In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
from datetime import timedelta
import glob
import geopandas as gpd
import geobr

C:\Users\linde\AppData\Roaming\Python\Python311\site-packages\geopandas\_compat.py:154: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  set_use_pygeos()
C:\Users\linde\AppData\Local\Temp\ipykernel_24008\22088038.py:9: DeprecationWarning: GeoPandas is set to use PyGEOS over Shapely. PyGEOS support is deprecatedand will be removed in GeoPandas 1.0, released in the Q1 of 2024. Please migrate to Shapely 2.0 (https://geopandas.org/en/stable/docs/user_guide/pygeos_to_shapely.html).
  import geopandas as gpd


In [3]:
plt.style.use("seaborn-v0_8-whitegrid")
plt.rcParams['axes.grid'] = False
plt.rcParams['font.size'] = '14'
plt.rcParams["font.family"] = "Times New Roman"

In [4]:
# variables
threshold_pr_max = 600 #[mm]
threshold_pr_min = 0 #[mm]

# Data processing

In [5]:
df_data = pd.read_hdf('./1 - Organized data gauge/BRAZIL/BRAZIL_DAILY_1961_2024.h5', key = 'table_data', encoding = 'utf-8')
df_data

,gauge_code,datetime,rain_mm
0,00047000,1961-01-01,0.0
1,00047000,1961-01-02,0.0
2,00047000,1961-01-03,0.0
3,00047000,1961-01-04,0.0
4,00047000,1961-01-05,0.0
...,...,...,...
1076753,S717,2021-12-27,0.0
1076754,S717,2021-12-28,0.0
1076755,S717,2021-12-29,0.0
1076756,S717,2021-12-30,0.0


In [6]:
df_info = pd.read_hdf('./1 - Organized data gauge/BRAZIL/BRAZIL_DAILY_1961_2024.h5', key = 'table_info', encoding = 'utf-8')
df_info = df_info[df_info['gauge_code'].isin(df_data['gauge_code'].unique())]
df_info

,name_station,gauge_code,city,state,responsible,source,state_abbreviation,lat,long
18981,SELVIRIA | S717,S717,SELVIRIA,MATO GROSSO DO SUL,INMET,INMET,MS,-20.351389,-51.430278
18980,SANTA RITA DO PARDO | S716,S716,SANTA RITA DO PARDO,MATO GROSSO DO SUL,INMET,INMET,MS,-21.305889,-52.820375
18979,RIBAS DO RIO PARDO | S715,S715,RIBAS DO RIO PARDO,MATO GROSSO DO SUL,INMET,INMET,MS,-20.466694,-53.763028
18978,PEDRO GOMES | S714,S714,PEDRO GOMES,MATO GROSSO DO SUL,INMET,INMET,MS,-18.072778,-54.548889
18977,NOVA ANDRADINA | S713,S713,NOVA ANDRADINA,MATO GROSSO DO SUL,INMET,INMET,MS,-22.078611,-53.465833
...,...,...,...,...,...,...,...,...,...
4,MARUDA,00047005,MARAPANIM,PARÁ,ANA,HIDROWEB,PA,-0.633600,-47.658300
3,PRIMAVERA,00047004,PRIMAVERA,PARÁ,ANA,HIDROWEB,PA,-0.929400,-47.099400
2,CURUÇA,00047003,CURUÇA,PARÁ,ANA,HIDROWEB,PA,-0.737500,-47.853600
1,SALINÓPOLIS,00047002,SALINÓPOLIS,PARÁ,ANA,HIDROWEB,PA,-0.623100,-47.353600


In [7]:
df_complete_info = pd.merge(df_data, df_info, on='gauge_code', how = 'inner').sort_values('lat', ascending = True)
# .dropna(how='any')
df_complete_info

,gauge_code,datetime,rain_mm,name_station,city,state,responsible,source,state_abbreviation,lat,long
123526277,A899,2022-06-20,0.0,Santa Vitoria do Palmar - Barra do Chui | A899,SANTA VITORIA DO PALMAR - BARRA DO CHUI,RIO GRANDE DO SUL,INMET,INMET,RS,-33.742222,-53.372222
123526411,A899,2022-11-01,0.0,Santa Vitoria do Palmar - Barra do Chui | A899,SANTA VITORIA DO PALMAR - BARRA DO CHUI,RIO GRANDE DO SUL,INMET,INMET,RS,-33.742222,-53.372222
123526412,A899,2022-11-02,0.0,Santa Vitoria do Palmar - Barra do Chui | A899,SANTA VITORIA DO PALMAR - BARRA DO CHUI,RIO GRANDE DO SUL,INMET,INMET,RS,-33.742222,-53.372222
123526413,A899,2022-11-03,0.0,Santa Vitoria do Palmar - Barra do Chui | A899,SANTA VITORIA DO PALMAR - BARRA DO CHUI,RIO GRANDE DO SUL,INMET,INMET,RS,-33.742222,-53.372222
123526414,A899,2022-11-04,0.0,Santa Vitoria do Palmar - Barra do Chui | A899,SANTA VITORIA DO PALMAR - BARRA DO CHUI,RIO GRANDE DO SUL,INMET,INMET,RS,-33.742222,-53.372222
...,...,...,...,...,...,...,...,...,...,...,...
114852986,08460003,2017-01-08,0.0,ÁGUA FRIA,UIRAMUTA,RORAIMA,ANA,HIDROWEB,RR,4.642800,-60.496400
114852987,08460003,2017-01-09,0.0,ÁGUA FRIA,UIRAMUTA,RORAIMA,ANA,HIDROWEB,RR,4.642800,-60.496400
114852988,08460003,2017-01-10,0.0,ÁGUA FRIA,UIRAMUTA,RORAIMA,ANA,HIDROWEB,RR,4.642800,-60.496400
114852975,08460003,2016-12-28,41.2,ÁGUA FRIA,UIRAMUTA,RORAIMA,ANA,HIDROWEB,RR,4.642800,-60.496400


In [8]:
df_data_filtered = df_data[(df_data['rain_mm'] >= threshold_pr_min) & (df_data['rain_mm'] <= threshold_pr_max)]
df_info_filtered = df_info[df_info['gauge_code'].isin(df_data_filtered['gauge_code'].unique())]

In [ ]:
print(len(df_complete_info) - len(df_data_filtered), 'data points were removed due to the precipitation threshold.')


4683 data points were removed due to the precipitation threshold.


In [9]:
print((len(df_info) - len(df_info_filtered)), 'gauges were removed due to the precipitation threshold.')

0 gauges were removed due to the precipitation threshold.


In [10]:
print((len(df_data) - len(df_data_filtered))/len(df_data)*100,"%")

0.00378829781618747 %


In [11]:
print((len(df_info) - len(df_info_filtered))/len(df_info)*100,"%")

0.0 %


In [12]:
df_data_filtered.to_hdf('./1 - Organized data gauge/BRAZIL/DATASETS/BRAZIL_DAILY_1961_2024_CLEANED.h5', key='table_data', mode='w', complevel=9, append = False, complib='zlib', encoding='utf-8')
df_info_filtered.to_hdf('./1 - Organized data gauge/BRAZIL/DATASETS/BRAZIL_DAILY_1961_2024_CLEANED.h5', key='table_info', mode='r+', complevel=9, append = False, complib='zlib', encoding='utf-8')

In [13]:
df_info_filtered = pd.read_hdf('./1 - Organized data gauge/BRAZIL/DATASETS/BRAZIL_DAILY_1961_2024_CLEANED.h5', key='table_info', encoding='utf-8')
df_info_filtered

,name_station,gauge_code,city,state,responsible,source,state_abbreviation,lat,long
18981,SELVIRIA | S717,S717,SELVIRIA,MATO GROSSO DO SUL,INMET,INMET,MS,-20.351389,-51.430278
18980,SANTA RITA DO PARDO | S716,S716,SANTA RITA DO PARDO,MATO GROSSO DO SUL,INMET,INMET,MS,-21.305889,-52.820375
18979,RIBAS DO RIO PARDO | S715,S715,RIBAS DO RIO PARDO,MATO GROSSO DO SUL,INMET,INMET,MS,-20.466694,-53.763028
18978,PEDRO GOMES | S714,S714,PEDRO GOMES,MATO GROSSO DO SUL,INMET,INMET,MS,-18.072778,-54.548889
18977,NOVA ANDRADINA | S713,S713,NOVA ANDRADINA,MATO GROSSO DO SUL,INMET,INMET,MS,-22.078611,-53.465833
...,...,...,...,...,...,...,...,...,...
4,MARUDA,00047005,MARAPANIM,PARÁ,ANA,HIDROWEB,PA,-0.633600,-47.658300
3,PRIMAVERA,00047004,PRIMAVERA,PARÁ,ANA,HIDROWEB,PA,-0.929400,-47.099400
2,CURUÇA,00047003,CURUÇA,PARÁ,ANA,HIDROWEB,PA,-0.737500,-47.853600
1,SALINÓPOLIS,00047002,SALINÓPOLIS,PARÁ,ANA,HIDROWEB,PA,-0.623100,-47.353600
